DC, for the Employment data analysis Please do the Data preparatin as following:
 
Get the following fields from the CICdata
 
customer_id, TIN, run_date, occupationdesc, occupationstatusdesc, TradeName, employment_duration_in_days (DateHiredTo or run_date - DateHiredFrom), PSICdesc, AnnualMonthlyIndicator, Currency, GrossIncome, max_sanctioned_amt_24m, max_creditcard_limit_24m, max_financed_amt_24m, tot_granted_contracts_cnt_24m, tot_granted_contracts_amt_12m, max_sanctioned_amt_12m, Max_CreditCard_limit_12m, max_financed_amt_12m, tot_granted_contracts_cnt_12m, tot_granted_contracts_amt_12m
 
Get the self declared employment details, application_date,  and MaxSalary information from our internal Salary scaling table using the CustomerID or TIN and application_date = run_date
 
Get the gender, age, region, province of the customer from demographic table using customer_id or TIN and application_date = run_date
 
Get the following fields from the Credolab tables using the customer_id or mobile number and application_date = run date:
Device Price
Device Manufacturer (Apple/ Samsung/ Oppo etc.)
Device Model
Device Age (usually derived by the application_date - earliest apps installation date)
number of job search apps installed (linkedin, jobstreet, glint etc.)
number of professional apps installed (office, gdrive etc)
number of ecommerce apps installed (lazada, shopee, shein, zalora, temu etc.)
number of ride hailing apps installed (grab, moveit etc.)
number of finance apps installed (Banking apps, FinTech apps, eWallet apps)
number of Risky apps (gambling apps, crypto apps, payday loan apps)
number of OTT apps (Netflix, PrimeVideo etc.)
Sowndarya: may be you can share your earlier apps categorisation work with DC for this. To keep things simple, just give him your final app categorisation mapping.

# <div align="center" style="color: #C64242;">**Employement Data with CIC Features and Customer Demographic**</div>

## <div style="color: #D9534F;"> Declaring the library </div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
import re
from fuzzywuzzy import process
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## <div style="color: #C64242; font-family: Monotype Corsiva;"><b> Main Query Part </b> </div>

In [2]:
%%time

# getting the customer demographic information from loan master table and employment information from cic employment data table

sq ="""# Loan Master 
with loanmaster as --- data from loan master for customer demographic details
(select distinct customerId, digitalLoanAccountId, loanAccountNumber, startApplyDateTime, natureofwork,industryDescription, incomeMultiple, employmentstatus,credolabScore,city,
       barangay, province, postalCode, age, monthlyIncome,  tinNumber, gender, dateOfBirth, email, mobileNo,firstName, middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity,  tsa_onboarding_time,new_loan_type
 from prj-prod-dataplatform.risk_credit_mis.loan_master_table),
employmentdata as ---- data from employment table for employment related information
(SELECT distinct
  digitalLoanAccountId, 
  crifApplicationId, 
  customerId,   
  AnnualMonthlyIndicator, 
  Currency, 
  DateHiredFrom, 
  DateHiredTo, 
  GrossIncome,
  CAST(
    CASE 
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)/12, 0)
      ELSE 0 
    END AS INT64
  ) AS MonthlyIncomecalculated,
  CAST(
    CASE 
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)*12, 0)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      ELSE 0 
    END AS INT64
  ) AS AnnualIncomecalculated,  
  Occupation,  
  OccupationDesc,
  OccupationStatus,
  OccupationStatusDesc,
  TIN,
  PhoneNumber,
  PSIC, 
  REGEXP_REPLACE(PSICDesc, r'^\d+\s*-\s*', '') AS PSICDesc ,
  row_number() over (partition by digitalLoanAccountId order by digitalLoanAccountId ) as rnk
FROM prj-prod-dataplatform.risk_credit_cic_data.employment_data)
select  ---joining both the above tables
lm.customerId, lm.digitalLoanAccountId, loanAccountNumber,
       startApplyDateTime, natureofwork, industryDescription,
       incomeMultiple, employmentstatus, credolabScore, city,
       barangay, province, postalCode, age, monthlyIncome,
       tinNumber, gender, dateOfBirth, email, mobileNo, firstName,
       middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity, tsa_onboarding_time,
       new_loan_type, ed.digitalLoanAccountId edprimarykey, crifApplicationId,
       AnnualMonthlyIndicator, DateHiredFrom,
       DateHiredTo, GrossIncome, MonthlyIncomecalculated, AnnualIncomecalculated, Occupation, OccupationDesc,
       OccupationStatus, OccupationStatusDesc, TIN, PhoneNumber,
       PSIC, PSICDesc
from loanmaster lm 
inner join (select * from employmentdata where rnk = 1) ed on ed.digitalLoanAccountId = lm.digitalLoanAccountId"""

dfdemograph_employment = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID f4346d01-9290-4224-af64-99a8f065bd21 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
CPU times: total: 25.5 s
Wall time: 3min 29s


In [3]:
dfdemograph_employment.shape

(274142, 44)

In [4]:
sfdc = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\Selected_features_dc.csv")
sfdc.shape

(42313, 11)

In [5]:
sfdc.columns

Index(['digitalLoanAccountId', 'max_amt_granted_24M', 'max_amt_granted_12M',
       'max_credit_limit_12M', 'max_financed_amt_12M', 'max_credit_limit_24M',
       'max_financed_amt_24M', 'tot_granted_contracts_amt_24M',
       'granted_contracts_cnt_24M', 'tot_granted_contracts_amt_12M',
       'granted_contracts_cnt_12M'],
      dtype='object')

In [6]:
dfdemograph_employment.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'startApplyDateTime', 'natureofwork', 'industryDescription',
       'incomeMultiple', 'employmentstatus', 'credolabScore', 'city',
       'barangay', 'province', 'postalCode', 'age', 'monthlyIncome',
       'tinNumber', 'gender', 'dateOfBirth', 'email', 'mobileNo', 'firstName',
       'middleName', 'lastName', 'custLoanJourneyZipCode',
       'custLoanJourneyProvince', 'custLoanJourneyCity', 'tsa_onboarding_time',
       'new_loan_type', 'edprimarykey', 'crifApplicationId',
       'AnnualMonthlyIndicator', 'DateHiredFrom', 'DateHiredTo', 'GrossIncome',
       'MonthlyIncomecalculated', 'AnnualIncomecalculated', 'Occupation',
       'OccupationDesc', 'OccupationStatus', 'OccupationStatusDesc', 'TIN',
       'PhoneNumber', 'PSIC', 'PSICDesc'],
      dtype='object')

In [7]:
mergeddf = dfdemograph_employment.merge(sfdc, on = 'digitalLoanAccountId', how = 'left' )

In [8]:
mergeddf.shape

(274142, 54)

In [9]:
mergeddf.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\Incomedata_20240808_v1.xlsx", sheet_name='Data')

In [10]:
mergeddf.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'startApplyDateTime', 'natureofwork', 'industryDescription',
       'incomeMultiple', 'employmentstatus', 'credolabScore', 'city',
       'barangay', 'province', 'postalCode', 'age', 'monthlyIncome',
       'tinNumber', 'gender', 'dateOfBirth', 'email', 'mobileNo', 'firstName',
       'middleName', 'lastName', 'custLoanJourneyZipCode',
       'custLoanJourneyProvince', 'custLoanJourneyCity', 'tsa_onboarding_time',
       'new_loan_type', 'edprimarykey', 'crifApplicationId',
       'AnnualMonthlyIndicator', 'DateHiredFrom', 'DateHiredTo', 'GrossIncome',
       'MonthlyIncomecalculated', 'AnnualIncomecalculated', 'Occupation',
       'OccupationDesc', 'OccupationStatus', 'OccupationStatusDesc', 'TIN',
       'PhoneNumber', 'PSIC', 'PSICDesc', 'max_amt_granted_24M',
       'max_amt_granted_12M', 'max_credit_limit_12M', 'max_financed_amt_12M',
       'max_credit_limit_24M', 'max_financed_amt_24M',
       'tot_granted_